In [2]:
import pandas as pd
import numpy as np

from quantfreedom.indicators.talib_ind import from_talib

import yfinance as yf

In [3]:
# Fetch historical data

# determine variable 'symbol' for fetching many tickets simultaneously (at same time)
# symbol = [
#     'BTC-USD', 'ETH-USD', 'XRP-USD', 'BCH-USD', 'LTC-USD'#,
# ]
# """    'BNB-USD', 'EOS-USD', 'XLM-USD', 'XMR-USD', 'ADA-USD',
#     'QBIT-USD', 'TSLA', 'AMZN', 'AAPL', 'DGTX-USD', 'BTG-USD', 
#     'LINK-USD', 'DOGE-USD', 'SOL-USD', 'TRX-USD', 'ATOM-USD',
#     'TON-USD', 'OKB-USD', 'GLD', 'GIS', 'PG', 'MRK', 'AMT', 'BMY',
#     'CVX', 'S', 'CVS', 'HD', 'PUK'
# #]
# """


symbol = ['BTC-USD']

# "Attach" dataset prices (dataframe) to variable ohlcv and at the same time download dataset from Yahoo Finance 
ohlcv = yf.download(symbol, interval="5m", period="60d")

# "Attach" only closing prices from dataset to variable 'price'
prices = ohlcv['Close']

# "Attach" only High prices from dataset to variable 'high_price'
high_price = ohlcv['High']

# Same for low
low_price = ohlcv['Low']

[*********************100%***********************]  1 of 1 completed


In [5]:
#Renaming columns for TA-Lib
ohlcv.rename(columns={'Open': 'open', 'High': 'high', 'Low': 'low', 'Adj Close': 'close', 'Volume': 'volume'}, inplace=True)

In [7]:
rsi_ind = from_talib(
    func_name='rsi',
    df_prices=ohlcv,
    cart_product=False,
    combos=False,
)

rsi_ind

RSI_timeperiod,14
Datetime,
2023-02-04 00:00:00+00:00,NaN
2023-02-04 00:05:00+00:00,NaN
2023-02-04 00:10:00+00:00,NaN
2023-02-04 00:15:00+00:00,NaN
2023-02-04 00:20:00+00:00,NaN
...,...
2023-04-04 02:10:00+00:00,55.33
2023-04-04 02:15:00+00:00,56.10
2023-04-04 02:20:00+00:00,58.08


In [ ]:
def rsi_closed_bellow(
        ohlcv,
        indicator,

):
        if ohlcv(closed) iloc-1 (meaning 1 bar after), True:
       
                else False

In [1]:
def eval_is_below(
    ind_data: pdFrame,    # 
    user_args: Optional[Union[list[int, float], int, float, Array1d]] = None,
    ind_results: Optional[Union[pdFrame, pdSeries]] = None,
    df_prices: Optional[pdFrame] = None,
    cand_ohlc: Optional[str] = None,
) -> pdFrame:
    if not isinstance(ind_data, pdFrame):
        raise ValueError("Data must be a dataframe with multindex")

    ind_data_values = ind_data.values
    pd_col_names = list(ind_data.columns.names) + [
        ind_data.columns.names[0].split("_")[0] + "_is_below"
    ]
    pd_multind_tuples = ()

    if isinstance(user_args, (list, Array1d)):
        eval_array = np.empty(
            (ind_data.shape[0], ind_data.shape[1] * len(user_args)), dtype=np.bool_
        )
        if not all(isinstance(x, (int, float, np.int_, np.float_)) for x in user_args):
            raise ValueError("user_args must be a list of ints or floats")
        eval_array_counter = 0
        for col in range(ind_data.shape[1]):
            temp_array = ind_data_values[:, col]
            for eval_col in range(user_args.size):
                eval_array[:, eval_array_counter] = np.where(
                    temp_array < user_args[eval_col], True, False
                )
                eval_array_counter += 1

                pd_multind_tuples = pd_multind_tuples + (
                    ind_data.columns[col] + (user_args[eval_col],),
                )

    elif isinstance(user_args, (int, float)):
        eval_array = np.where(ind_data_values < user_args, True, False)

        for col in range(ind_data.shape[1]):
            pd_multind_tuples = pd_multind_tuples + (
                ind_data.columns[col] + (user_args,),
            )

    elif isinstance(df_prices, pdFrame):
        eval_array = np.empty(ind_data, dtype=np.bool_)
        if cand_ohlc == None or cand_ohlc.lower() not in (
            "open",
            "high",
            "low",
            "close",
        ):
            raise ValueError(
                "cand_ohlc must be open, high, low or close when sending price data"
            )
        price_values = getattr(df_prices, cand_ohlc).values
        if not all(isinstance(x, (np.int_, np.float_)) for x in price_values):
            raise ValueError("price data must be ints or floats")
        for col in range(ind_data.shape[1]):
            eval_array[:, col] = np.where(
                ind_data_values[:, col] < price_values, True, False
            )

            pd_multind_tuples = pd_multind_tuples + (
                ind_data.columns[col] + (cand_ohlc,),
            )

    elif isinstance(ind_results, pdSeries):
        eval_array = np.empty_like(ind_data, dtype=np.bool_)
        price_values = ind_results.values
        if not all(isinstance(x, (np.int_, np.float_)) for x in price_values):
            raise ValueError("price data must be ints or floats")
        for col in range(ind_data.shape[1]):
            eval_array[:, col] = np.where(
                ind_data_values[:, col] > price_values, True, False
            )

            pd_multind_tuples = pd_multind_tuples + (
                ind_data.columns[col] + ("something",),
            )
    else:
        raise ValueError(
            "user_args must be a list of ints or floats or int or float or you need to send price data"
        )

    return pd.DataFrame(
        eval_array,
        index=ind_data.index,
        columns=pd.MultiIndex.from_tuples(
            tuples=list(pd_multind_tuples),
            names=pd_col_names,
        ),
    )

NameError: name 'pdFrame' is not defined